In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
pwd

'/Users/mayankkapadia/Downloads'

In [3]:
list_of_csv=['Tuesday-WorkingHours.pcap_ISCX.csv','Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv','Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv','Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv','Monday-WorkingHours.pcap_ISCX.csv','Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv','Tuesday-WorkingHours.pcap_ISCX.csv','Wednesday-workingHours.pcap_ISCX.csv']
combined_df=pd.DataFrame()
l=[]
for x in list_of_csv:
    df=pd.read_csv(x, on_bad_lines='skip')
    median_values_each_class=df.groupby(' Label')['Flow Bytes/s'].median().reset_index()
    median_values_each_class.columns=['Label','Median']
    median_values_each_class.set_index('Label',inplace=True)
    median_mapping=median_values_each_class.to_dict()
    df['Flow Bytes/s']=df['Flow Bytes/s'].fillna(df[' Label'].map(median_mapping['Median']))
    print(df['Flow Bytes/s'].isnull().value_counts())
    l.append(df)
combined_df = pd.concat(l, ignore_index=True)

Flow Bytes/s
False    445909
Name: count, dtype: int64
Flow Bytes/s
False    225745
Name: count, dtype: int64
Flow Bytes/s
False    286467
Name: count, dtype: int64
Flow Bytes/s
False    286467
Name: count, dtype: int64
Flow Bytes/s
False    529918
Name: count, dtype: int64
Flow Bytes/s
False    288602
Name: count, dtype: int64
Flow Bytes/s
False    445909
Name: count, dtype: int64
Flow Bytes/s
False    692703
Name: count, dtype: int64


In [4]:
combined_df.to_csv('modified_daata.csv',index=False)

In [5]:
import pandas as pd

In [2]:
df=pd.read_csv('/Users/mayankkapadia/Downloads/modified_daata.csv')

In [ ]:
df[' Label'].unique()

In [4]:
df[' Timestamp']=pd.to_datetime(df[' Timestamp'], errors='coerce',infer_datetime_format=True)

In [5]:
len(df)

3201720

In [6]:
df[' Timestamp']=df[' Timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [7]:
df[' Timestamp'].isnull().value_counts()

False    2671802
True      529918
Name:  Timestamp, dtype: int64

In [8]:
df=df.dropna()

In [9]:
len(df[' Timestamp'].unique())

1485

In [10]:
len(df[' Source IP'].unique())

13485

In [11]:
len(df)

2671802

In [12]:
source_df=df[[' Source IP',' Source Port']]
source_df=source_df.drop_duplicates()

In [13]:
source_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372053 entries, 0 to 3201709
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0    Source IP    372053 non-null  object
 1    Source Port  372053 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 8.5+ MB


In [14]:
source_df[' Source Port']=source_df[' Source Port'].astype(str)

In [15]:
source_df['surrogate_key'] = range(1, len(source_df) + 1)
source_df['source_ip_prt']=source_df[' Source IP']+':'+source_df[' Source Port']

In [16]:
len(source_df['surrogate_key'].unique())

372053

In [17]:
source_df.columns

Index([' Source IP', ' Source Port', 'surrogate_key', 'source_ip_prt'], dtype='object')

In [18]:
source_df.to_csv('Source_IP.csv',index=False)

In [19]:
destination_df=df[[' Destination IP',' Destination Port']]
destination_df=destination_df.drop_duplicates()

In [20]:
destination_df[' Destination Port']=destination_df[' Destination Port'].astype(str)

In [21]:
destination_df['surrogate_key'] = range(1, len(destination_df) + 1)
destination_df['dest_ip_port']=destination_df[' Destination IP']+":"+destination_df[' Destination Port']
destination_df.to_csv('Destination_IP.csv',index=False)

In [22]:
import hashlib

In [23]:
flow_df=df[['Flow ID',' Flow Duration','Flow Bytes/s',' Flow Packets/s', ' Flow IAT Mean',' Fwd IAT Std',' Fwd IAT Max', ' Fwd IAT Min']]
flow_df=flow_df.drop_duplicates()
def generate_hash(row):
    # Convert the row to a string representation
    row_str = str(tuple(row.values))
    # Generate a hash using SHA256
    hash_object = hashlib.sha256(row_str.encode())
    return hash_object.hexdigest()

# Apply the hash function to create a new 'Merged_Col'
flow_df['Merged_Col'] = flow_df.apply(generate_hash, axis=1)

In [24]:
flow_df['surrogate_key'] = range(1, len(flow_df) + 1)


In [25]:
len(flow_df['Merged_Col'].unique())

1921280

In [26]:
len(flow_df)

1921280

In [27]:
flow_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1921280 entries, 0 to 3201719
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Flow ID          object 
 1    Flow Duration   int64  
 2   Flow Bytes/s     float64
 3    Flow Packets/s  float64
 4    Flow IAT Mean   float64
 5    Fwd IAT Std     float64
 6    Fwd IAT Max     float64
 7    Fwd IAT Min     float64
 8   Merged_Col       object 
 9   surrogate_key    int64  
dtypes: float64(6), int64(2), object(2)
memory usage: 161.2+ MB


In [28]:
flow_df.to_csv('Flow.csv',index=False)

In [29]:
df[' Total Fwd Packets']=df[' Total Fwd Packets'].astype(float)
df[' Total Backward Packets']=df[' Total Backward Packets'].astype(float)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2671802 entries, 0 to 3201719
Data columns (total 85 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Flow ID                       object 
 1    Source IP                    object 
 2    Source Port                  int64  
 3    Destination IP               object 
 4    Destination Port             int64  
 5    Protocol                     int64  
 6    Timestamp                    object 
 7    Flow Duration                int64  
 8    Total Fwd Packets            float64
 9    Total Backward Packets       float64
 10  Total Length of Fwd Packets   float64
 11   Total Length of Bwd Packets  float64
 12   Fwd Packet Length Max        float64
 13   Fwd Packet Length Min        float64
 14   Fwd Packet Length Mean       float64
 15   Fwd Packet Length Std        float64
 16  Bwd Packet Length Max         float64
 17   Bwd Packet Length Min        float64
 18   Bwd Packet Length Mea

In [31]:
packet_df=df[[' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std','Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std',' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',' Average Packet Size']]
packet_df=packet_df.drop_duplicates()
def generate_hash(row):
    # Convert the row to a string representation
    row_str = str(tuple(row.values))
    # Generate a hash using SHA256
    hash_object = hashlib.sha256(row_str.encode())
    return hash_object.hexdigest()

# Apply the hash function to create a new 'Merged_Col'
packet_df['Merged_Col'] = packet_df.apply(generate_hash, axis=1)

In [32]:
import hashlib

In [34]:
packet_df['surrogate_key'] = range(1, len(packet_df) + 1)
packet_df.to_csv('Packet.csv',index=False)

In [35]:
len(packet_df)

299932

In [36]:
packet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299932 entries, 0 to 3201713
Data columns (total 20 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0    Total Fwd Packets            299932 non-null  float64
 1    Total Backward Packets       299932 non-null  float64
 2   Total Length of Fwd Packets   299932 non-null  float64
 3    Total Length of Bwd Packets  299932 non-null  float64
 4    Fwd Packet Length Max        299932 non-null  float64
 5    Fwd Packet Length Min        299932 non-null  float64
 6    Fwd Packet Length Mean       299932 non-null  float64
 7    Fwd Packet Length Std        299932 non-null  float64
 8   Bwd Packet Length Max         299932 non-null  float64
 9    Bwd Packet Length Min        299932 non-null  float64
 10   Bwd Packet Length Mean       299932 non-null  float64
 11   Bwd Packet Length Std        299932 non-null  float64
 12   Min Packet Length            299932 non-nu

In [37]:
len(packet_df['Merged_Col'].unique())

299932

In [38]:
flag_df=df[['Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags','FIN Flag Count', ' SYN Flag Count', ' RST Flag Count',
       ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
       ' CWE Flag Count', ' ECE Flag Count']]
flag_df=flag_df.drop_duplicates()

def generate_hash(row):
    # Convert the row to a string representation
    row_str = str(tuple(row.values))
    # Generate a hash using SHA256
    hash_object = hashlib.sha256(row_str.encode())
    return hash_object.hexdigest()

# Apply the hash function to create a new 'Merged_Col'
flag_df['Merged_Col'] = flag_df.apply(generate_hash, axis=1)

In [39]:
flag_df['surrogate_key'] = range(1, len(flag_df) + 1)
flag_df.to_csv('Flag.csv',index=False)

In [40]:
len(flag_df['Merged_Col'].unique())

14

In [41]:
len(flag_df)

14

In [42]:
flag_df.to_csv('Flag.csv',index=False)

In [43]:
flag_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14 entries, 0 to 3127085
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Fwd PSH Flags    14 non-null     int64 
 1    Bwd PSH Flags   14 non-null     int64 
 2    Fwd URG Flags   14 non-null     int64 
 3    Bwd URG Flags   14 non-null     int64 
 4   FIN Flag Count   14 non-null     int64 
 5    SYN Flag Count  14 non-null     int64 
 6    RST Flag Count  14 non-null     int64 
 7    PSH Flag Count  14 non-null     int64 
 8    ACK Flag Count  14 non-null     int64 
 9    URG Flag Count  14 non-null     int64 
 10   CWE Flag Count  14 non-null     int64 
 11   ECE Flag Count  14 non-null     int64 
 12  Merged_Col       14 non-null     object
 13  surrogate_key    14 non-null     int64 
dtypes: int64(13), object(1)
memory usage: 1.6+ KB


In [44]:
protocol_df=df[[ ' Protocol']]
protocol_df=protocol_df.drop_duplicates()

In [45]:
protocol_df['Protocol Name']=['TCP','Reserved','UDP']

In [46]:
protocol_df['Protocol Description']=["This is a connection-oriented protocol which ensures that the transmission or transfer of data between devices on the network takes place in a sequential manner and with controlled errors in a reliable way","Some bits or fields in some network protocols - for example, the headers of IP packets-are set aside for future use or special purposes and shall not be altered during normal transmission", "This protocol is connectionless and therefore does not guarantee delivery, but it's faster and less reliable than TCP; it always finds its application when time is of essence, such as video playing or online gaming."]

In [47]:
protocol_df

,Protocol,Protocol Name,Protocol Description
0,6,TCP,This is a connection-oriented protocol which e...
55,0,Reserved,Some bits or fields in some network protocols ...
56,17,UDP,This protocol is connectionless and therefore ...


In [48]:
protocol_df['surrogate_key'] = range(1, len(protocol_df) + 1)
protocol_df.to_csv('Protocol.csv',index=False)

In [49]:
df[' Timestamp'].isnull().value_counts()

False    2671802
Name:  Timestamp, dtype: int64

In [50]:
date_df=df[[' Timestamp']]
date_df=date_df.drop_duplicates()
date_df.dropna(inplace=True)

In [51]:
date_df.isnull().value_counts()

 Timestamp
False         1485
dtype: int64

In [52]:
date_df[' Timestamp'] = pd.to_datetime(date_df[' Timestamp'])
date_df['day']=date_df[' Timestamp'].dt.day
date_df['month']=date_df[' Timestamp'].dt.month
date_df['year']=date_df[' Timestamp'].dt.year
date_df['time']=date_df[' Timestamp'].dt.time

In [53]:
date_df

,Timestamp,day,month,year,time
0,2017-04-07 08:54:00,7,4,2017,08:54:00
18,2017-04-07 08:55:00,7,4,2017,08:55:00
40,2017-04-07 08:53:00,7,4,2017,08:53:00
49,2017-04-07 08:56:00,7,4,2017,08:56:00
62,2017-04-07 08:57:00,7,4,2017,08:57:00
...,...,...,...,...,...
3024032,2017-05-07 05:06:00,7,5,2017,05:06:00
3024142,2017-05-07 05:07:00,7,5,2017,05:07:00
3024232,2017-05-07 05:08:00,7,5,2017,05:08:00
3038626,2017-05-07 05:10:00,7,5,2017,05:10:00


In [54]:
date_df['surrogate_key'] = range(1, len(date_df) + 1)
date_df.to_csv('Date.csv',index=False)

In [55]:
date_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1485 entries, 0 to 3135308
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0    Timestamp     1485 non-null   datetime64[ns]
 1   day            1485 non-null   int64         
 2   month          1485 non-null   int64         
 3   year           1485 non-null   int64         
 4   time           1485 non-null   object        
 5   surrogate_key  1485 non-null   int64         
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 81.2+ KB


In [56]:
date_df

,Timestamp,day,month,year,time,surrogate_key
0,2017-04-07 08:54:00,7,4,2017,08:54:00,1
18,2017-04-07 08:55:00,7,4,2017,08:55:00,2
40,2017-04-07 08:53:00,7,4,2017,08:53:00,3
49,2017-04-07 08:56:00,7,4,2017,08:56:00,4
62,2017-04-07 08:57:00,7,4,2017,08:57:00,5
...,...,...,...,...,...,...
3024032,2017-05-07 05:06:00,7,5,2017,05:06:00,1481
3024142,2017-05-07 05:07:00,7,5,2017,05:07:00,1482
3024232,2017-05-07 05:08:00,7,5,2017,05:08:00,1483
3038626,2017-05-07 05:10:00,7,5,2017,05:10:00,1484


In [52]:
import pandas as pd

In [53]:
mdf=pd.read_csv('modified_daata.csv')

In [57]:
fact_df=df.copy()

In [58]:
ans=(fact_df==df).any()

In [59]:
fact_df[' Source Port']=fact_df[' Source Port'].astype(str)

In [60]:
fact_df['Source_IP_Port']=fact_df[[' Source IP',' Source Port']].agg(':'.join,axis=1)
fact_df['Source_ID_FK']=fact_df['Source_IP_Port'].map(source_df.set_index(['source_ip_prt'])['surrogate_key'])

In [61]:
destination_df.columns

Index([' Destination IP', ' Destination Port', 'surrogate_key',
       'dest_ip_port'],
      dtype='object')

In [62]:
fact_df[' Destination Port']=fact_df[' Destination Port'].astype(str)
fact_df['Destination_IP_Port']=fact_df[' Destination IP']+":"+fact_df[' Destination Port']
fact_df['Dest_IP_FK']=fact_df['Destination_IP_Port'].map(destination_df.set_index(['dest_ip_port'])['surrogate_key'])

In [63]:
fact_df['Protocol_FK']=fact_df[' Protocol'].map(protocol_df.set_index([' Protocol'])['surrogate_key'])

In [64]:
fact_df['Timestamp_FK']=fact_df[' Timestamp'].map(date_df.set_index([' Timestamp'])['surrogate_key'])

In [65]:
def generate_packet_hash(row):
    # Select only the desired columns for hashing
    selected_cols = row[[' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std','Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std',' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',' Average Packet Size']]
    
    # Convert the selected columns to a string representation
    row_str = str(tuple(selected_cols.values))
    
    # Generate a hash using SHA256
    hash_object = hashlib.sha256(row_str.encode())
    return hash_object.hexdigest()

# Apply the hash function to the packet columns
fact_df['Merged_Col'] = fact_df.apply(generate_packet_hash, axis=1)

# Map the hash to the corresponding surrogate key from packet_df
fact_df['Packet_FK'] = fact_df['Merged_Col'].map(packet_df.set_index('Merged_Col')['surrogate_key'])

In [66]:
fact_df.head()

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,Idle Min,Label,Source_IP_Port,Source_ID_FK,Destination_IP_Port,Dest_IP_FK,Protocol_FK,Timestamp_FK,Merged_Col,Packet_FK
0,192.168.10.3-192.168.10.5-88-49182-6,192.168.10.5,49182,192.168.10.3,88,6,2017-04-07 08:54:00,640,7.0,4.0,...,0.0,BENIGN,192.168.10.5:49182,1,192.168.10.3:88,1,1,1,a4a086ab6838154eaf0d52f3c6c9c3e00160b7a91bf1ec...,1
1,192.168.10.3-192.168.10.5-88-49183-6,192.168.10.5,49183,192.168.10.3,88,6,2017-04-07 08:54:00,900,9.0,4.0,...,0.0,BENIGN,192.168.10.5:49183,2,192.168.10.3:88,1,1,1,02364cdfc3db4afd88bc43cfc4d015babe75f99a0405c6...,2
2,192.168.10.3-192.168.10.5-88-49184-6,192.168.10.5,49184,192.168.10.3,88,6,2017-04-07 08:54:00,1205,7.0,4.0,...,0.0,BENIGN,192.168.10.5:49184,3,192.168.10.3:88,1,1,1,9bd933d56fb1991c9c7df92fb46c6f1b77e4c2db20bd87...,3
3,192.168.10.3-192.168.10.5-88-49185-6,192.168.10.5,49185,192.168.10.3,88,6,2017-04-07 08:54:00,511,7.0,4.0,...,0.0,BENIGN,192.168.10.5:49185,4,192.168.10.3:88,1,1,1,44bc2b3b8650e23199d4f89e64e6f387b791cacdd901e6...,4
4,192.168.10.3-192.168.10.5-88-49186-6,192.168.10.5,49186,192.168.10.3,88,6,2017-04-07 08:54:00,773,9.0,4.0,...,0.0,BENIGN,192.168.10.5:49186,5,192.168.10.3:88,1,1,1,9a2c77eceaa0ff591d68940855b039b1d92ee668a38223...,5


In [67]:
def generate_flow_hash(row):
    # Select only the desired columns for hashing
    selected_cols_flow = row[['Flow ID', ' Flow Duration', 'Flow Bytes/s', ' Flow Packets/s', 
                         ' Flow IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min']]
    
    # Convert the selected columns to a string representation
    row_str = str(tuple(selected_cols_flow.values))
    
    # Generate a hash using SHA256
    hash_object = hashlib.sha256(row_str.encode())
    return hash_object.hexdigest()

# Apply the hash function to the flow columns
fact_df['Merged_Col'] = fact_df.apply(generate_flow_hash, axis=1)

# Map the hash to the corresponding surrogate key from flow_df
fact_df['Flow_FK'] = fact_df['Merged_Col'].map(flow_df.set_index('Merged_Col')['surrogate_key'])

In [68]:
def generate_flag_hash(row):
    # Select only the desired columns for hashing
    selected_cols = row[['Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags','FIN Flag Count', ' SYN Flag Count', ' RST Flag Count',
       ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
       ' CWE Flag Count', ' ECE Flag Count']]
    
    # Convert the selected columns to a string representation
    row_str = str(tuple(selected_cols.values))
    
    # Generate a hash using SHA256
    hash_object = hashlib.sha256(row_str.encode())
    return hash_object.hexdigest()

# Apply the hash function to the flag columns
fact_df['Merged_Col'] = fact_df.apply(generate_flag_hash, axis=1)

fact_df['Flag_FK'] = fact_df['Merged_Col'].map(flag_df.set_index('Merged_Col')['surrogate_key'])

In [69]:
fact_df.columns

Index(['Flow ID', ' Source IP', ' Source Port', ' Destination IP',
       ' Destination Port', ' Protocol', ' Timestamp', ' Flow Duration',
       ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Pa

In [70]:
for x in fact_df.columns:
    if fact_df[x].isnull().any()==True:
        print(x)


In [71]:
new_fact_df=fact_df[['Source_ID_FK', 'Dest_IP_FK', 'Protocol_FK', 'Timestamp_FK', 'Packet_FK', 'Flow_FK', 'Flag_FK',' Label']]

In [72]:
new_fact_df.head()

,Source_ID_FK,Dest_IP_FK,Protocol_FK,Timestamp_FK,Packet_FK,Flow_FK,Flag_FK,Label
0,1,1,1,1,1,1,1,BENIGN
1,2,1,1,1,2,2,1,BENIGN
2,3,1,1,1,3,3,1,BENIGN
3,4,1,1,1,4,4,1,BENIGN
4,5,1,1,1,5,5,1,BENIGN


In [73]:
new_fact_df.to_csv('fact_table.csv')

In [74]:
len(new_fact_df)

2671802

In [ ]:
fact_table_df=pd.read_csv('fact_table.csv')
fact_table_df

In [76]:
new_fact_df_1=new_fact_df[:900000]
new_fact_df_1.to_csv('fact_table_1.csv')

In [77]:
new_fact_df_2=new_fact_df[900000:1800000]
new_fact_df_2.to_csv('fact_table_2.csv')

In [78]:
new_fact_df_3=new_fact_df[1800000:]
new_fact_df_3.to_csv('fact_table_3.csv')

In [79]:
len(flow_df)

1921280

In [81]:
flow_df_1=flow_df[:400000]
flow_df_1.to_csv('flow_1.csv')

In [82]:
flow_df_2=flow_df[400000:800000]
flow_df_3=flow_df[800000:1200000]
flow_df_4=flow_df[1200000:1600000]
flow_df_5=flow_df[1600000:]

In [83]:
flow_df_2.to_csv('flow_2.csv')

In [84]:
flow_df_3.to_csv('flow_3.csv')
flow_df_4.to_csv('flow_4.csv')
flow_df_5.to_csv('flow_5.csv')